In [24]:
import nibabel as nib
import numpy as np

fixed_nib = nib.load("/home/mathdugre/.cache/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz").get_fdata()
nib.Nifti1Image(np.zeros(fixed_nib.shape), np.eye(4)).to_filename("blank.nii.gz")
nib.Nifti1Image(np.full(fixed_nib.shape, np.finfo(np.float64).max), np.eye(4)).to_filename("full.nii.gz")

In [28]:
def mutual_information(hgram):
    """ Mutual information for joint histogram
    """
    # Convert bins counts to probability values
    pxy = hgram / float(np.sum(hgram))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    px_py = px[:, None] * py[None, :] # Broadcast to multiply marginals
    # Now we can do the calculation using the pxy, px_py 2D arrays
    nzs = pxy > 0 # Only non-zero pxy values contribute to the sum
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

In [2]:
import itk

fixed_itk = itk.imread("/home/mathdugre/.cache/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz", itk.D)
blank_itk = itk.imread("blank.nii.gz", itk.D)
full_itk = itk.imread("full.nii.gz", itk.D)
sanity_itk = itk.imread("/home/mathdugre/papers/fast-interpolate/dataset/ds004513/derivatives/flint/antsRegistration/binary64/sub-s003/Warped.nii.gz", itk.D)

In [ ]:
import itk.itkMutualInformationImageToImageMetricPython

TransformType = itk.TranslationTransform[itk.D, 3]
transform = TransformType.New()

MetricType = itk.MutualInformationImageToImageMetric[
    type(fixed_itk), type(blank_itk)
]
metric = MetricType.New()
metric.SetFixedImage(fixed_itk)
metric.SetMovingImage(blank_itk)

metric.SetNumberOfSpatialSamples(100)
metric.SetFixedImageStandardDeviation(0.4)
metric.SetMovingImageStandardDeviation(0.4)

# Initialize the metric
# metric.Initialize()

# Compute the similarity value
similarity_value = metric.GetValue(transform.GetParameters())
print(f"Similarity Value: {similarity_value:.16e}")


: 

In [27]:
MetricType = itk.ANTSNeighborhoodCorrelationImageToImageMetricv4[
    type(fixed_itk), type(full_itk)
]
metric = MetricType.New()
metric.SetFixedImage(fixed_itk)
metric.SetMovingImage(full_itk)

# Initialize the metric
metric.Initialize()

# Compute the similarity value
similarity_value = metric.GetValue()
print(f"Similarity Value: {similarity_value:.16e}")


Similarity Value: -1.0000000000000000e+00


In [36]:
MetricType = itk.ANTSNeighborhoodCorrelationImageToImageMetricv4[
    type(fixed_itk), type(sanity_itk)
]
metric = MetricType.New()
metric.SetFixedImage(fixed_itk)
metric.SetMovingImage(sanity_itk)

# Initialize the metric
metric.Initialize()

# Compute the similarity value
similarity_value = metric.GetValue()
print(f"Similarity Value: {similarity_value:.16e}")


Similarity Value: -8.6242698995606148e-01


In [39]:
blank_nib = nib.load("blank.nii.gz").get_fdata()
full_nib = nib.load("full.nii.gz").get_fdata()
sanity_nib = nib.load("/home/mathdugre/papers/fast-interpolate/dataset/ds004513/derivatives/flint/antsRegistration/binary64/sub-s003/Warped.nii.gz").get_fdata()

In [32]:
hist_2d, x_edges, y_edges = np.histogram2d(
    fixed_nib.ravel(),
    blank_nib.ravel(),
    bins=32
)
mutual_information(hist_2d)

np.float64(-3.0741979043401205e-16)

In [33]:
hist_2d, x_edges, y_edges = np.histogram2d(
    fixed_nib.ravel(),
    full_nib.ravel(),
    bins=32
)
mutual_information(hist_2d)

np.float64(-3.0741979043401205e-16)

In [37]:
import skimage as ski

ski.metrics.normalized_mutual_information(fixed_nib, blank_nib, bins=32)

np.float64(1.0)

In [38]:
ski.metrics.normalized_mutual_information(fixed_nib, full_nib, bins=32)

divide by zero encountered in divide
invalid value encountered in divide


np.float64(nan)

In [41]:
-(ski.metrics.normalized_mutual_information(fixed_nib, sanity_nib, bins=32) - 1)

np.float64(-0.3580196755123446)